In [1]:
import numpy as np
import pandas as pd
import copy
import random

In [2]:
# Classroom Class which is unique by weekDay 
class Classroom():
    def __init__(self, roomNumber, courseClasses, weekDay):
        self.roomNumber = roomNumber
        self.courseClasses = courseClasses
        self.day = weekDay
    
    def findClassesForTeacher(self, teacher):
        classes = []
        for i in range(len(self.courseClasses)):
            if (self.courseClasses[i].teacher == teacher):
                classes.append(self.courseClasses[i])
        return classes
    
    def findClassesForSection(self, section):
        classes = []
        for i in range(len(self.courseClasses)):
            if (self.courseClasses[i].section == section):
                classes.append(self.courseClasses[i])
        return classes
    
    def findClassesForCourse(self,course):
        classes = []
        for i in range(len(self.courseClasses)):
            if (self.courseClasses[i].course == course):
                classes.append(self.courseClasses[i])
        return classes
    
    def classExistsAlready(self, Class):
        #print("In function: ", end = " ")
        for courseClass in self.courseClasses:
            if (courseClass.room == Class.room and courseClass.course == Class.course and courseClass.type == Class.type):
                if (courseClass.section == Class.section):
         #           print("true")
                    return True
        #print("false")
        return False
    


In [3]:
#Class in classroom 
class Class():
    def __init__(self, course, section, duration, teacher, startTime, roomNumbers, Type):
        self.course = course
        self.section = section
        self.duration = int(duration)
        self.teacher = teacher
        self.startTime = startTime
        self.roomNumbers = []
        for roomNumber in roomNumbers:
            if (roomNumber != ","):
                self.roomNumbers.append(roomNumber)
        #print(self.roomNumbers, "Length: ", len(self.roomNumbers))
        self.type = Type
        self.room = None
    
    def checkCollision(self, other):
        if (self.startTime < other.startTime and (self.startTime + (self.duration*100)) > other.startTime):
            return True
        elif (other.startTime < self.startTime and (other.startTime + (other.duration*100)) > self.startTime):
            return True
        else:
            return False
        
        
    def _print(self):
        print(self.course)
        print("Start time: ", self.startTime, " End time: ", self.startTime + (100*self.duration), " Duration: ", self.duration)
    
    def equal(self, other):
        if (other.course == self.course and other.type == self.type and other.section == self.section):
            return True
        else:
            return False
        

In [4]:
#Hard Constraints
class HardConstraints():

    def __init__(self, classrooms):
        self.classrooms = classrooms
        
    #No teacher can hold two classes at the same time
    def asynchronusTeaching(self, teacher):
        #print("Length is: ", len(self.classrooms))
        teacherClasses = []
        for i in range(len(self.classrooms)):
            classes = self.classrooms[i].findClassesForTeacher(teacher)
            for j in range(len(classes)):
                teacherClasses.append(classes[j])
        for i in range(len(teacherClasses)):
            for j in range(len(teacherClasses)):
                if (i != j):
                    #teacherClasses[i]._print()
                    #teacherClasses[j]._print()
                    if (teacherClasses[i].checkCollision(teacherClasses[j])):
                        return 0
        return 1
    #No section can listen for two classes at the same time
    def asynchronusClassTaking(self, section):
        sectionClasses = []
        for i in range(len(self.classrooms)):
            classes = self.classrooms[i].findClassesForSection(section)
            for j in range(len(classes)):
                sectionClasses.append(classes[j])
        for i in range(len(sectionClasses)):
            for j in range(len(sectionClasses)):
                if (i != j):
                    #sectionClasses[i]._print()
                    #sectionClasses[j]._print()
                    if (sectionClasses[i].checkCollision(sectionClasses[j])):
                        return 0
        return 1
    #No classroom can receive two classes at the same time   
    def asynchronusClassroomBooking(self):
        for i in range(len(self.classrooms)):
            classes = self.classrooms[i].courseClasses
            for j in range(len(classes)):
                for k in range(len(classes)):
                    if (j != k):
                        if (classes[j].checkCollision(classes[k])):
                            return 0
        return 1
    #No teacher can hold three consecutive classes
    def consecutiveTeaching(self, teacher):
        teacherClasses = []
        for i in range(len(self.classrooms)):
            classes = copy.deepcopy(self.classrooms[i].findClassesForTeacher(teacher))
            for j in range(len(classes)):
                teacherClasses.append(classes[j])
        for i in range(len(teacherClasses)):
            for j in range(len(teacherClasses)- 1 - i):
                if (teacherClasses[j].startTime > teacherClasses[j+1].startTime):
                    teacherClasses[j], teacherClasses[j+1] = teacherClasses[j+1], teacherClasses[j]
        consecutiveCount = 1
        for i in range(len(teacherClasses) - 1):
            if (teacherClasses[i].startTime + teacherClasses[i].duration == teacherClasses[i+1].startTime):
                consecutiveCount = consecutiveCount + 1
            else:
                consecutiveCount = 1
        if (consecutiveCount < 3):
            return 1
        else:
            return 0
    #There will be no class before 8:30 am and after 5:00 pm.
    def badTiming(self):
        classes = []
        for i in range(len(self.classrooms)):
            Classes = self.classrooms[i].courseClasses
            for j in range(len(Classes)):
                classes.append(Classes[j])
                
        for i in range(len(classes)):
            if (classes[i].startTime < 830 or classes[i].startTime > 1700):
                return 0
            elif (classes[i].startTime + classes[i].duration > 1700):
                return 0
        return 1
    
    def _print(self):
        for classroom in self.classrooms:
            for Class in classroom.courseClasses:
                print(Class.room, Class.startTime, Class.course, Class.type, Class.section, Class.teacher)
    

In [5]:
#functions for soft constraints
class SoftConstraints():
    
    def __init__(self, classrooms):
        self.classrooms = classrooms
    #No section can hold three consecutive classes.
    def consecutiveSectionTeaching(self, section):
        sectionClasses = []
        for i in range(len(self.classrooms)):
            classes = copy.deepcopy(self.classrooms[i].findClassesForSection(section))
            for j in range(len(classes)):
                sectionClasses.append(classes[j])
        for i in range(len(sectionClasses)):
            for j in range(len(classes)- 1 - i):
                if (sectionClasses[j].startTime > sectionClasses[j+1].startTime):
                    sectionClasses[j], sectionClasses[j+1] = sectionClasses[j+1], sectionClasses[j]
        consecutiveCount = 1
        for i in range(len(classes) - 1):
            #sectionClasses[i]._print()
            #sectionClasses[i+1]._print()
            if (sectionClasses[i].startTime + sectionClasses[i].duration == sectionClasses[i+1].startTime):
                consecutiveCount = consecutiveCount + 1
            else:
                consecutiveCount = 1
        if (consecutiveCount < 3):
            return 1
        else:
            return 0
    #There will be no class from 1-2 on Friday.
    def jummahBreak(self):
        classes = []
        for i in range(len(self.classrooms)):
            temp = self.classrooms[i].courseClasses
            for j in range(len(temp)):
                classes.append(temp[j])
        for i in range(len(classes)):
            #classes[i]._print()
            if (classes[i].startTime < 1300 and (classes[i].startTime + classes[i].duration) > 1300):
                return 0
            elif (classes[i].startTime > 1300 and classes[i].startTime < 1400):
                return 0
        return 1
    #A subject having multiple forms, such as lectures and labs, the preferred order is: lecture and lab.
    def firstClassThenLab(self):
        for i in range(len(self.classrooms)):
            classes = self.classrooms[i].courseClasses
            for j in range(len(classes)):
                if (classes[j].type == "Lecture"):
                    for k in range(len(classes)):
                        if (j != k):
                            if (classes[k].course == classes[j].course):
                                if (classes[k].type == "Lab"):
                                    if (k < j):
                                        return 0
        return 1
                                    
    

In [6]:
# time table class 
class TimeTable():
    
    def __init__(self, classes, classrooms, teachers, sections):
        self.classes = classes
        self.days = {"Monday": [],"Tuesday": [], "Wednesday": [], "Thursday": [], "Friday": []}
        self.teachers = teachers
        self.sections = sections
        self.score = None
        for classroom in classrooms:
            if (classroom.day ==  "Monday"):
                self.days["Monday"].append(classroom)
            elif (classroom.day == "Tuesday"):
                self.days["Tuesday"].append(classroom)
            elif (classroom.day == "Wednesday"):
                self.days["Wednesday"].append(classroom)
            elif (classroom.day == "Thursday"):
                self.days["Thursday"].append(classroom)
            elif (classroom.day == "Friday"):
                self.days["Friday"].append(classroom)
    
    def addClassToRandomRoom(self, Class, Day):
        classrooms = self.days[Day]
        #print("Length of roomNumbers: ", len(Class.roomNumbers))
        index = random.randint(0,len(Class.roomNumbers)-1)
        #print(Class.roomNumbers)
        #print(index, Class.roomNumbers[index])
        Class.room = Class.roomNumbers[index]
        #print("Here", Class.room)
        #print("Length of classrooms: ", len(classrooms))
        for classroom in classrooms:
            #print(classroom.roomNumber, Class.room, "----")
            if (classroom.roomNumber == Class.room):
                if (classroom.classExistsAlready(Class) == False):
                    classroom.courseClasses.append(Class)
                    return True
                else: 
                    #print("It was true")
                    return False
        return False
                
    def addClass(self,Class,roomNumber):
        classrooms = self.days[Day]
        Class.room = Class.roomNumbers[roomNumber]
        for classroom in classrooms:
            if (classroom.roomNumber == Class.room):
                classroom.courseClasses.append(Class)
                
    def _print(self):
        for day in self.days:
            print(day)
            for classroom in self.days[day]:
                classes = classroom.courseClasses
                for Class in classes:
                    print(Class.course, Class.room, end=" ")
            print("\n")
            
    def _printTeachers(self):
        for day in self.days:
            print(day)
            for classroom in self.days[day]:
                classes = classroom.courseClasses
                for Class in classes:
                    print(Class.course, Class.room, Class.teacher, end=" ")
            print("\n")
    
    def _printSections(self):
        for day in self.days:
            print(day)
            for classroom in self.days[day]:
                classes = classroom.courseClasses
                for Class in classes:
                    print(Class.course, Class.room, Class.section, end=" ")
            print("\n")
        
    def _printType(self):
        for day in self.days:
            print(day)
            for classroom in self.days[day]:
                classes = classroom.courseClasses
                for Class in classes:
                    print(Class.course, Class.room, Class.type, end=" ")
            print("\n")
    
    def _printTimeOfClass(self):
        for day in self.days:
            print(day)
            for classroom in self.days[day]:
                classes = classroom.courseClasses
                for Class in classes:
                    Class._print()
    
    def _printProperTimeTable(self):
        for day in self.days:
            for classroom in self.days[day]:
                classes = classroom.courseClasses
                for Class in classes:
                    print(Class.course, Class.startTime, Class.duration, Class.room, Class.teacher, Class.section, day)
                    
            

In [7]:
#read dataset
df = pd.read_csv("dataset2.csv")

In [8]:
#printing dataset
print(df)

       Name           Course Section  Duration         Classroom
0    Valeed          Network       D         1      2,4,6,8,9,10
1     Bilal              Web       A         1    1,3,5,6,8,9,10
2      Omar  Data Structures       C         1       1,5,6,7,8,9
3    Valeed              Web       C         3  3,4,5,6,7,8,9,10
4   Shayaan         Database       A         1      1,5,6,8,9,10
5   Mustafa         Database       B         1       3,4,6,2,7,8
6     Bilal         Database       C         1      1,3,4,5,9,10
7   Furrukh         Database       D         1   4,5,2,6,1,3,8,9
8   Shayaan         Database       A         3    1,2,3,7,8,9,10
9   Mustafa         Database       B         3   4,5,2,6,1,3,8,9
10   Valeed         Database       C         3    1,2,3,7,8,9,10
11  Shaheer         Database       D         3  1,2,3,5,6,8,9,10
12     Omar          Network       A         1         2,3,4,5,6
13    Bilal          Network       B         1   4,5,2,6,1,3,8,9
14     Omar          Netw

In [9]:
# getting all rooms 
roomNumbers = []
rooms = list(df['Classroom'])


In [10]:
# getting proper format of rooms
for room in rooms:
    temp = room.split(",")
    for number in temp:
        if number not in roomNumbers:
            roomNumbers.append(number)
#print(roomNumbers)

In [11]:
# creating classrooms through room numbers
classrooms = []
for room in roomNumbers:
    classrooms.append(Classroom(room,[],"Monday"))
    classrooms.append(Classroom(room,[],"Tuesday"))
    classrooms.append(Classroom(room,[],"Wednesday"))
    classrooms.append(Classroom(room,[],"Thursday"))
    classrooms.append(Classroom(room,[],"Friday"))


In [12]:
# storing data in variables for making class
names = list(df['Name'])
courses = list(df['Course'])
sections = list(df['Section'])
durations = list(df['Duration'])
rooms = list(df['Classroom'])

uniqueNames = np.unique(names)
uniqueSections = np.unique(sections)

In [13]:
# creating classes 
classes = []
for i in range(len(names)):
    if (durations[i] == 1):
        temp = Class(courses[i],sections[i],durations[i],names[i],0,rooms[i].split(","),"Lecture")
    else:
        temp = Class(courses[i],sections[i],durations[i],names[i],0,rooms[i],"Lab")
    classes.append(temp)

In [14]:
#creaitng the first time table 
timeTable = TimeTable(classes,classrooms, uniqueNames, uniqueSections)

In [15]:
# genetic algorithm 
class GeneticAlgorithm():
    
    def __init__(self, timeTable, mutation_rate):
        self.mutation_rate = mutation_rate
        self.population = []
        self.timeSlots = None
        for i in range(100):
         #   print(i)
            temp = copy.deepcopy(timeTable)
            self.population.append(temp)
        self.scores = None
    
    #sort for replacing half of bad population with childs of good population
    def sort(self):
        for i in range(len(self.population)):
            for j in range(len(self.population) - 1 - i):
                if (self.population[i].score < self.population[i+1].score):
                    self.population[i], self.population[i+1] = self.population[i+1], self.population[i]
    #populate randomely
    def populateRandom(self):
        num = 0
        for timeTable in self.population:
           # print(num)
            num = num + 1
            n = int(len(timeTable.classes)/len(timeTable.days))
            #print(n)
            total = 0
            for day in timeTable.days:
                for i in range(n):
                    someBool = False
                    classIndex = random.randint(0,len(timeTable.classes)-1)
                    while (someBool == False):
                        classIndex = random.randint(0,len(timeTable.classes)-1)
                        #print(classIndex, len(timeTable.classes))
                        someBool = timeTable.addClassToRandomRoom(timeTable.classes[classIndex],day)
                        #print(timeTable.classes[classIndex]._print())
                    #print("Out of while loop")
            timeSlots = []
            start = 830
            for i in range(16):
                timeSlots.append(start)
                if (i%2 == 0):
                    start = start + 70
                else:
                    start = start + 30
            #print(timeSlots)
            self.timeSlots = timeSlots
            for day in timeTable.days:
                for classroom in timeTable.days[day]:
                    for Class in classroom.courseClasses:
                        if (Class.duration == 1):
                            index = random.randint(0,len(timeSlots)-1)
                        else:
                            index = random.randint(0,len(timeSlots)-5)
                        Class.startTime = timeSlots[index]
    
    '''def populateRandom2(self):
        num = 0
        #for timeTable in self.population:
        timeTable = self.population[0]
        
        n = int(len(timeTable.classes)/len(timeTable.days))
        #print(n)
        total = 0
        for day in timeTable.days:
            for i in range(n):
                someBool = False
                classIndex = random.randint(0,len(timeTable.classes)-1)
                while (someBool == False):
                    classIndex = random.randint(0,len(timeTable.classes)-1)
                    #print(classIndex, len(timeTable.classes))
                    someBool = timeTable.addClassToRandomRoom(timeTable.classes[classIndex],day)
                    #print(timeTable.classes[classIndex]._print())
                #print("Out of while loop")
        
        for i in range(len(self.population)):
            self.population[i] = copy.deepcopy(timeTable)
        
        for timeTable in self.population:
           # print(num)
           # num = num + 1
            timeSlots = []
            start = 830
            for i in range(16):
                timeSlots.append(start)
                if (i%2 == 0):
                    start = start + 70
                else:
                    start = start + 30
            self.timeSlots = timeSlots
            for day in timeTable.days:
                for classroom in timeTable.days[day]:
                    for Class in classroom.courseClasses:
                        if (Class.duration == 1):
                            index = random.randint(0,len(timeSlots)-1)
                        else:
                            index = random.randint(0,len(timeSlots)-5)
                        Class.startTime = timeSlots[index]
        
        '''
    # fitness function     
    def fitness(self):
        hardScore = []
        softScore = []
        score = []
        for i in range(len(self.population)):
            hardScore.append([])
            softScore.append([])
        for i in range(len(self.population)):
            count = 0
            teachers = self.population[i].teachers
            for teacher in teachers:
                for day in self.population[i].days:
                    hard = HardConstraints(self.population[i].days[day])
                    count = count + hard.asynchronusTeaching(teacher)
            if (count == (len(teachers)*len(self.population[i].days))):
                hardScore[i].append(1)
            else:
                hardScore[i].append(0)
            
            
            count = 0
            sections = self.population[i].sections
            for section in sections:
                for day in self.population[i].days:
                    hard = HardConstraints(self.population[i].days[day])
                    count = count + hard.asynchronusClassTaking(section)
            if (count == (len(sections)*len(self.population[i].days))):
                hardScore[i].append(1)
            else:
                hardScore[i].append(0)
            
            
            count = 0
            for day in self.population[i].days:
                hard = HardConstraints(self.population[i].days[day])
                count = count + hard.asynchronusClassroomBooking()
            if (count == len(self.population[i].days)):
                hardScore[i].append(1)
            else:
                hardScore[i].append(0)
                
            count = 0
            for teacher in teachers:
                for day in self.population[i].days:
                    hard = HardConstraints(self.population[i].days[day])
                    count = count + hard.consecutiveTeaching(teacher)
            if (count == (len(teachers)*len(self.population[i].days))):
                hardScore[i].append(1)
            else:
                hardScore[i].append(0)
            
            count = 0
            for day in self.population[i].days:
                hard = HardConstraints(self.population[i].days[day])
                count = count + hard.badTiming()
            if (count == (len(self.population[i].days))):
                hardScore[i].append(1)
            else:
                hardScore[i].append(0)
                
            count = 0
            for section in sections:
                for day in self.population[i].days:
                    soft = SoftConstraints(self.population[i].days[day])
                    count = count + soft.consecutiveSectionTeaching(section)
            if (count == (len(sections)*len(self.population[i].days))):
                softScore[i].append(1)
            else:
                softScore[i].append(0)
            
            count = 0
            soft = SoftConstraints(self.population[i].days['Friday'])
            count = count + soft.jummahBreak()
            if (count == 1):
                softScore[i].append(1)
            else:
                softScore[i].append(0)
                
            count = 0
            for day in self.population[i].days:
                soft = SoftConstraints(self.population[i].days[day])
                count = count + soft.firstClassThenLab()
            if (count == len(self.population[i].days)):
                softScore[i].append(1)
            else:
                softScore[i].append(0)
        
        for i in range(len(self.population)):
            total = 0
            for j in hardScore[i]:
                if (j == 1):
                    total = total + 10
            for j in softScore[i]:
                if (j == 1):
                    total = total + 1
            score.append(total)
       # print("Elite timetables: ")
        for i in range(len(self.population)):
            self.population[i].score = score[i]
        self.scores = score
        
    def mutate(self, day):
        mutation = np.random.normal()
        if (mutation <= self.mutation_rate):
            index = random.randint(0,len(day)-1)
           # while(len(day[index].courseClasses) == 0):
            #    index = random.randint(0,len(day)-1)
            if (len(day[index].courseClasses) == 0):
                return day
                #print("Chaning index...")
            classIndex = random.randint(0,len(day[index].courseClasses)-1)
            time = random.randint(0,len(self.timeSlots)-1)
            day[index].courseClasses[classIndex].startTime = self.timeSlots[time]
        return day
        
    # crossover function 
    def crossover(self):
        i = 0
        week = ["Monday","Tuesday","Wednesday","Thursday","Friday"]
        while (i < len(self.population) - 1):
            getSplit = random.randint(0,len(week)-1)
            for index in range(getSplit):
                getSplit2 = random.randint(0,len(self.population[i].days[week[index]])-1)
                classrooms = copy.deepcopy(self.population[i].days[week[index]][:getSplit2])
                self.population[i].days[week[index]][:getSplit2] = copy.deepcopy(self.population[i+1].days[week[index]][:getSplit2])
                self.population[i+1].days[week[index]][:getSplit2] = copy.deepcopy(classrooms)
                self.mutate(self.population[i].days[week[index]])
                self.mutate(self.population[i+1].days[week[index]])
            i = i + 2
    # remove bad population and replace with childs of good population
    def eradicate(self,old,new):
        length = len(old)
        new[length:] = copy.deepcopy(old[length:])
        return new
    
    # breeding procedure for new child population
    def breed(self):
        self.sort()
       # print("Sorted 1")
        oldPopulation = copy.deepcopy(self.population)
        self.crossover()
       # print("Crossover Done")
        self.sort()
       # print("Sorted 2")
        newPopulation = self.eradicate(oldPopulation, self.population)
       # print("Bad Eradicated")
        self.population = copy.deepcopy(newPopulation)
        
    # check if all constraints are matched or not 
    def solutionFound(self):
        
        print("Max score rn: ", max(self.scores))
        for timeTable in self.population:
            if (timeTable.score == 53):
                return timeTable
        
        return None
    
    # genetic algorithm run function 
    def run(self):
        self.populateRandom()
        self.fitness()
        generation = 0
        while(self.solutionFound() == None):
        #for i in range(2):
            self.breed()
            self.fitness()
            generation = generation + 1
            print("Generation: ", generation)
        print("Solution found")
        timeTable = self.solutionFound()
        print("All hard constraints fulfilled")
        print("All soft constraints except faculty meeting are fullfilled")
        return timeTable    
        

In [16]:
algo = GeneticAlgorithm(timeTable, 0.17)

In [17]:
timeTable = algo.run()

Max score rn:  33
Generation:  1
Max score rn:  43
Generation:  2
Max score rn:  43
Generation:  3
Max score rn:  33
Generation:  4
Max score rn:  33
Generation:  5
Max score rn:  33
Generation:  6
Max score rn:  33
Generation:  7
Max score rn:  33
Generation:  8
Max score rn:  33
Generation:  9
Max score rn:  33
Generation:  10
Max score rn:  33
Generation:  11
Max score rn:  33
Generation:  12
Max score rn:  33
Generation:  13
Max score rn:  33
Generation:  14
Max score rn:  33
Generation:  15
Max score rn:  43
Generation:  16
Max score rn:  43
Generation:  17
Max score rn:  43
Generation:  18
Max score rn:  43
Generation:  19
Max score rn:  42
Generation:  20
Max score rn:  43
Generation:  21
Max score rn:  43
Generation:  22
Max score rn:  33
Generation:  23
Max score rn:  33
Generation:  24
Max score rn:  43
Generation:  25
Max score rn:  42
Generation:  26
Max score rn:  42
Generation:  27
Max score rn:  43
Generation:  28
Max score rn:  43
Generation:  29
Max score rn:  43
Gener

In [18]:
timeTable._printProperTimeTable()

Data Structures 830 3 6 Ahmed B Monday
Web 1330 3 4 Valeed C Monday
Data Structures 1330 3 1 Ahmed D Monday
Database 1330 3 3 Valeed C Monday
Database 830 3 6 Shaheer D Monday
Database 1230 3 2 Valeed C Tuesday
Web 1300 3 8 Omar A Tuesday
Data Structures 1030 1 3 Bilal D Tuesday
Network 1130 1 6 Valeed D Wednesday
Data Structures 1030 1 9 Valeed A Wednesday
Web 1300 3 9 Valeed C Wednesday
Network 900 3 1 Ahmed B Wednesday
Web 1430 3 5 Shayaan B Wednesday
Web 1600 1 2 Valeed C Thursday
Data Structures 1000 3 2 Shaheer C Thursday
Data Structures 1000 1 7 Omar C Thursday
Database 1330 1 9 Bilal C Thursday
Database 1400 1 3 Furrukh D Thursday
Data Structures 1000 3 5 Valeed A Thursday
Web 1430 3 5 Shayaan B Thursday
Database 1530 3 1 Mustafa B Friday
Network 1230 3 5 Ahmed B Friday
Data Structures 930 3 4 Ahmed D Friday
Database 930 3 6 Shaheer D Friday
Database 1530 3 1 Mustafa B Friday
Network 1230 3 5 Ahmed B Friday
